참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, ISBN 978-8964211489, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengage Learning, ISBN 978-1439062203, 2012.

`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)<br>
Bring in `module`'s that would expand features of `python`. (This file may use just some of them.)<br>
`import` `module`s to extend `python` features (This example may use selected features only)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능 array, matrix
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능 linear algebra
import matplotlib.pyplot as plt  # 그래프 관련 기능 plotting
import scipy.integrate as si  # 적분 관련 기능 numerical integration
import sympy as sy  # 기호 연산 기능 symbolic procesing
import sympy.plotting as splot
import IPython.display as disp  # 웹페이지 표시 기능 python notebook

기호 연산 결과 표시 기능 초기화<br>Initialize symbolic result printing

In [ ]:
sy.init_printing()

## 예제 06.001<br>Ex 06.001


p. 239

### 문제에서 주어진 변수<br>Given parameters

#### 보의 길이<br>Length of the beam

In [ ]:
L_AB = sy.symbols('L_{AB}[m]', positive=True)

나중에 수치값을 계산하기 위해 보의 길이의 값을 dictionary 로 준비한다. (이를 **대입 딕셔너리** 라고 부르기로 하자)<br>Store the numerical value of the length in a dictionary to substitute later. (Let's call this a **substitution dictionary**)

In [ ]:
s_d = {L_AB: 4}

#### 하중<br>Load

AB 구간의 분포하중<br>The distributed load of interval AB

In [ ]:
w_AB = sy.Symbol('w_{AB}[N/m]', real=True)

대입 딕셔너리에 분포하중 값을 추가한다.<br>Add distributed load value to the substitution dictionary.

In [ ]:
s_d[w_AB] = -2000

#### 재료<br>Material

Young's modulus

In [ ]:
E = sy.Symbol('E[Pa]', positive=True)
s_d[E] = 200e9

#### 단면<br>Section

W 200 $\times$ 100 에 관련된 자료는 p.608 참고<br>See table for section W 200 $\times$ 100

In [ ]:
A, I, h = sy.symbols('A[m^{2}], I[m^{4}], h[m]', positive=True)
A_mm2 = A * 1e-6
I_mm4 = I * 1e-12
h_mm = h * 1e3

여러 값을 대입 딕셔너리에 추가하기 위해 `update()`메소드를 사용하였다.<br>To add multiple values to `s_d`, used `update()` method.

In [ ]:
s_d.update({A_mm2: 12700.0,
            I_mm4: 113e6,
            h_mm: 229,})

m 단위<br>in m

In [ ]:
s_d.update({A: s_d[A_mm2] * 1e-6,
            I: s_d[I_mm4] * 1e-12,
            h: s_d[h_mm] * 1e-3,})

준비된 값을 확인<br>Visualize the substitution dictionary

In [ ]:
s_d

문제에서 주어진 보의 자유물체도를 도시<br>Draw a free body diagram

In [ ]:
import os   # 운영체제 관련 기능 Operating Systems
import sys  # 시스템 관련 기능 Systems
# utils 폴더의 모듈을 import 할 수 있도록 준비
# add utils folder to sys.path to import
sys.path.append(os.path.abspath(os.path.join(os.pardir, 'utils')))
# 선도 관련 기능 diagrams
import draw_diagrams

points_list = [
    {'x_m': 0, 'text':'A'},
    {'x_m': L_AB.subs(s_d), 'text':'B'},
]

reaction_list = [
    {'x_m': L_AB.subs(s_d)},
]

dist_load_list = [
    {'x_begin_m': 0, 'x_end_m': L_AB.subs(s_d), 'text': 'w0'}
]

moment_list = [
    {'x_m': L_AB.subs(s_d), 'direction': 'cw', 'text': '', 'open':'left'},
]

draw_diagrams.draw_beam(
    L_AB.subs(s_d), 
    points_list, 
    reaction_list, 
    dist_load_list=dist_load_list, 
    moment_list=moment_list
)

### (1) 탄성선의 방정식<br>(1) Equation of the elastic curve

#### 각 지점의 x 좌표<br>x coordinate of each location

A 점에서 $x=0$ 으로 한다.<br>
At A, let $x=0$.

In [ ]:
x = sy.Symbol('x[m]', nonnegative=True)
x_A = 0
x_B = float(x_A + L_AB.subs(s_d))

#### x 지점에서의 굽힘모멘트<br>Bending moment at $x$

In [ ]:
M = sy.integrate(-w_AB, x, x)

In [ ]:
M

#### 굽힘모멘트와 처짐 사이의 관계<br>Bending moment ~ deflection relationship

p. 236 eq. 6.3b

$$
EI\nu''=EI\frac{d^2\nu}{dx^2}=M
$$

$\nu$ 와 그 미분을 준비<br>$\nu$ and its derivative

In [ ]:
EI_nu = sy.Function(r'EI\nu[Nm^{3}]')
# http://docs.sympy.org/latest/modules/solvers/ode.html#dsolve
# http://docs.sympy.org/latest/tutorial/calculus.html
EI_nu_pp = sy.Derivative(EI_nu(x), x, x)

In [ ]:
EI_nu(x)

In [ ]:
EI_nu_pp

미분방정식 등호의 왼쪽<br>
Left side of the differential equation

In [ ]:
left_side = EI_nu_pp
left_side

등호의 오른쪽<br>Right side of the differential equation

In [ ]:
right_side = M
right_side

eq. 6.3b

In [ ]:
defl_deq = sy.Eq(left_side, right_side )
defl_deq

미분방정식을 두번 적분한 결과<br>Integrating twice gives

In [ ]:
EI_nu_sol_eq = sy.dsolve(defl_deq, EI_nu(x))
EI_nu_sol_eq

위 결과에서 등호의 오른쪽은 다음과 같다.<br>Right side of the result above

In [ ]:
EI_nu_sol = EI_nu_sol_eq.rhs
EI_nu_sol

#### 경계조건<br>Boundary conditions

여기서 경계조건을 이용하여 적분 상수를 결정한다.<br>
Let's decide integration constants using boundary conditions.

경계조건은 2가지로 $x=L$ 에서 기울기와 처짐이 모두 0이다.<br>Thera are two boundary conditions; at $x=L$, both slope and defection are zero.

먼저, 기울기 조건을 적용한다.<br>First, apply slope boundary condition:

$$ \left. \frac{d\nu}{dx} \right\rvert _{x=L}=0 $$

In [ ]:
# SymPy Dev Team, Derivatives, Calculus, SymPy Tutorial, 2017 Jul 27, http://docs.sympy.org/latest/tutorial/calculus.html#derivatives
slope_eq = sy.Eq(sy.diff(EI_nu_sol, x).subs(x, L_AB), 0)

In [ ]:
slope_eq

경계조건 두가지 (처짐, 기울기) 가운데 처짐 조건을 적용한다.<br>Apply deflection boundary condition:

$$ \left. \nu \right\rvert _{x=L}=0 $$

In [ ]:
defl_eq = sy.Eq(EI_nu_sol.subs(x, L_AB), 0)

In [ ]:
defl_eq

위 두 방정식을 연립하여 풂<br>Solving simultaneous equations gives:

In [ ]:
const_sol = sy.solve((slope_eq, defl_eq))

In [ ]:
const_sol

#### 탄성선의 방정식<br>Equation of the elastic curve

적분상수를 $EI\nu$ 식에 대입함<br>Substituting integration constants to $EI\nu$ gives:

In [ ]:
nu = sy.simplify(EI_nu_sol.subs(const_sol[0])) / (E * I)

In [ ]:
nu

변위를 선도로 표시해본다.<br>Let's plot the deflection.

In [ ]:
splot.plot(-nu.subs(s_d)*1e3, (x, 0, x_B), 
           xlabel='x(m)', ylabel='$\\nu$(mm)')

### (2) 최대변위<br>(2) Maximum deflection

최대 처짐은 $x=0$ 지점에서 발생한다.<br>At $x=0$, the deflection is the maximum.

In [ ]:
nu.subs(x, 0)

mm 단위로는 다음과 같다.<br>In mm unit:

In [ ]:
nu.subs(s_d).subs(x, 0)*1e3

#### 굽힘응력<br>Bending stress

굽힘 모멘트에 의한 응력도 계산해 본다.<br>Let's calculate bending stress too.

In [ ]:
sigma = M / I * (h / 2)

In [ ]:
sigma

In [ ]:
splot.plot(sigma.subs(s_d)*1e-6, (x, 0, x_B), 
           xlabel='x(m)', ylabel='$\\sigma$(MPa)')

응력의 최대값은 $x=L$ 지점에서 발생한다.<br>At $x=L$, the bending stress is the maximum.

In [ ]:
sigma.subs(x, L_AB)

Pa 단위 응력 값은 다음과 같다.<br>Numerical value of the maximum stress in Pa unit:

In [ ]:
sigma.subs(x, L_AB).subs(s_d)

MPa 단위로는 다음과 같다.<br>In MPa unit:

In [ ]:
sigma.subs(x, L_AB).subs(s_d) * 1e-6